![monitor](monitor_pic.jpg)


My current computer monitor is getting a bit dated and my eyes have started to feel sore from looking at it all day. I want to buy a new computer monitor, but every time I get online to start searching I get overwhelmed by the number of monitors available. They come in a wide variety of sizes, widths, definitions, display types, port types, etc, and vary in price from under \\$100 to well over \\$1000. While I know that I want to upgrade my screen size to give me more working area, other options, like gaming features, are irrelevant to me. How on earth will I ever be able to decide which monitor is right for me?

This seems like a perfect opportunity to put some data analysis into action! For this project I am going to be acquiring feature data for a number of different computer monitors and running some product analysis across those features to help me make a data driven decision which monitor to purchase. 

### First Step: Get the data
Unfortunately, I was unable to find a downloadable preexisting dataset for computer monitors so I had to make the dataset myself. I looked through a few different websites that carried a wide variety of monitors from different manufacturers but the available data was limited to pretty basic specifications. For this project I wanted to use a dataset with a rich feature set in order to try to explore more subtle differences in features and try to tease out their value. LG is one of the brands that seem to be pretty appealing to me, and when I looked at their website I found that for each monitor they provided over 50 data points and it was in a format that wouldnt be too difficult to scrape. So, my first step of the first step is going to be building a set of functions to scrape, clean and format the data for one monitor into a dataframe. Then later I will use those functions across multiple monitors and compile them all into one dataset. 

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

### Monitor URL list
I am starting by extracting the URL for each monitor into a list

In [2]:
# extracting the list of monitor URLs to a list
results = requests.get('https://www.lg.com/us/monitors')
content = results.content
soup = BeautifulSoup(content, 'lxml')
monitors = soup.find_all('a', class_='ga-model-detail')

monitor_list = [monitor.get('href') for monitor in monitors]
edited_monitor_list = monitor_list[2:46]
reedited_list = []
for m in edited_monitor_list:
    if m not in reedited_list:
        reedited_list.append(m)
monitor_url_list = [monitor.rsplit('/',1)[1] for monitor in reedited_list]
base_url = 'https://www.lg.com/us/monitors/'
monitor_url_list = [base_url + monitor_extention for monitor_extention in monitor_url_list]

print(f'This is the format for our monitor URLs in the list: {monitor_url_list[0]}')



This is the format for our monitor URLs in the list: https://www.lg.com/us/monitors/lg-27up650-w-4k-uhd-monitor


### Individual monitor data extraction
Next step is building the functions that will extract and clean up the data for each monitor

In [3]:
# downloading the selected spec tags for a given monitor from the LG website

results = requests.get('https://www.lg.com/us/monitors/lg-32ul500-w-4k-uhd-monitor')
content = results.content
parser = BeautifulSoup(content, 'lxml')

table = list(parser.find_all('div', class_="tech-spacs"))

# the result is a list with 10 elements
# each element is a string with spec data for one aspect of the monitor

table_type = type(table)
print(f"returned object is a {table_type} with {len(table)} elements ")

returned object is a <class 'list'> with 10 elements 


In [4]:
# cleaning the first element and turning it into a list

first_table = table[0].text.replace('\n', ' ')
first_table = first_table.replace('\t', ' ')
first_table = first_table.strip()
first_table_listed = first_table.rsplit('   ')

In [5]:
# cleaning up the list

q_list = [element for element in first_table_listed if element != '']
q_header = q_list[0]
q_body = q_list[1:]
f = lambda q: q.strip()
mapped_q = list(map(f, q_body))

In [6]:
#separating the list into category & value lists and recombining as a dictionary

category = [mapped_q[i] for i in range(len(mapped_q)) if i%2 == 0]
value = [mapped_q[i] for i in range(len(mapped_q)) if i%2 != 0]
monitor_dict = dict(zip(category, value))

In [7]:
# putting the values into a dataframe with the row representing the monitor and columns are its attributes

df = pd.DataFrame(data=monitor_dict, index=[0])
df

,Size,Display Type,Response Time,Refresh Rate,Display Resolution,Color Gamut (Typ.),Color Depth (Number of Colors),Pixel Pitch (mm),Aspect Ratio,Resolution,Brightness,Contrast Ratio,Viewing Angle,Surface Treatment
0,"32""",VA,4ms (GtG at Faster),60Hz,4K UHD,DCI-P3 95% (CIE1976),1.07B,0.181x 0.181 mm,16:9,3840 x 2160,300cd/m²,3000:1,"178˚(R/L), 178˚(U/D)",Anti-Glare


In [8]:
# putting it all together and building functions for the transformation of each returned scraped elements

def download_monitor_specs(URL):
    """Import monitor specs and parse into list of strings"""
    
    from bs4 import BeautifulSoup
    import requests
    results = requests.get(URL)
    content = results.content
    parser = BeautifulSoup(content, 'lxml')
    list_of_strings = list(parser.find_all('div', class_="tech-spacs"))
    return list_of_strings
    
def clean_element(list_of_strings):
    """Reformat list of HTML strings and return a list of lists"""
    
    listed_element_lists = []
    for string in list_of_strings:
        string = string.text.replace('\n', ' ')
        string = string.replace('\t', ' ')
        string = string.strip()
        listed_string = string.rsplit('   ')
        listed_element_lists.append(listed_string)
    return listed_element_lists

def clean_listed_element(listed_element_lists):
    """Remove blank lines and strip empty spaces on list elements"""
    
    cleaned_element_list = []
    for list_ in listed_element_lists:
        listed_element = [element for element in list_ if element != '']
        listed_element = listed_element[1:]
        f = lambda q: q.strip()
        listed_element = list(map(f, listed_element))
        cleaned_element_list.append(listed_element)
    return cleaned_element_list

def list_to_dict(cleaned_element_list):
    """Separate list into categories and values then combine into dictionary"""
    
    categories = []
    values = []
    for element in cleaned_element_list:
        category = [element[i] for i in range(len(element)) if i%2 == 0]
        categories += category
        value = [element[i] for i in range(len(element)) if i%2 != 0]
        values+=value
    
    monitor_dict = dict(zip(categories, values))
    return monitor_dict


def dict_to_df(monitor_dict):
    """Convert dictionary of monitor specs into pandas DataFrame object"""
    import pandas as pd
    df = pd.DataFrame(data=monitor_dict, index=[0])
    return df



In [9]:
# testing out the function pipeline so far

url = 'https://www.lg.com/us/monitors/lg-32ul500-w-4k-uhd-monitor'

df = dict_to_df(
    list_to_dict(
    clean_listed_element(
    clean_element(
    download_monitor_specs(url)
    ))))

df.shape

(1, 52)

### Putting together secondary path for review page info retrieval 

In [10]:
# for price, model name and reviews it was easier to scrape from a different URL than the initial scrape

def access_review_page(url):
    """redirect monitor url to monitor review url"""
    monitor_review_url = url + '/reviews'
    
    return monitor_review_url

def extract_review_elements(monitor_review_url):
    """Pulls values from monitor review website and returns dict with 'col':'value' format"""
    results = requests.get(monitor_review_url)
    content = results.content
    soup = BeautifulSoup(content, 'lxml')
    
    values_to_extract = {
        'model_name':'mpn',
        'model_title':'name',
        'price':'price',
        'rating_count':"reviewCount",
        'avg_rating':"ratingValue"
    }
    
    for column in values_to_extract.keys():
        label = values_to_extract[column]
        extracted_html_value = soup.find('meta', {'itemprop':label})
        extracted_value = extracted_html_value.get('content')
        # update dictionary value with extracted value
        values_to_extract[column]=extracted_value
        
    return values_to_extract
        


# Putting it all together
This part uses the list of monitors to retrieve all the data for each monitor and compiles it all together in one pandas dataframe so that I can get to work cleaning the data, and later perform some (hopefully) insightful analysis on it. 

In [11]:
# Initializing a new dataframe outside the for loop to concatonate with
df = pd.DataFrame()

# looping through the URL for each monitor
for monitor in monitor_url_list:
    # building dictionary from values on the monitor page
    d = list_to_dict(
    clean_listed_element(
    clean_element(
    download_monitor_specs(monitor)
    )))
    # building dictionary from values on the monitor review page
    d2 = access_review_page(monitor)
    d2 = extract_review_elements(d2)
    # combining the dictionaries
    d2.update(d)
    # converting the dictionaries into a dataframe
    df2 = pd.DataFrame(data=[d2])
    # adding the monitor's dataframe row to the initial dataframe 
    df = pd.concat([df, df2])
    

In [12]:
# setting up an index
df = df.reset_index(drop=True)
df.shape

(20, 175)

# On to data manipulation!
Finally, I have a working dataframe, now its time to clean it up! Unfortunately there are only 19 monitors for my dataset, which doesnt give me a lot of comparison points to work with. However there are nearly 200 features! An awful lot of those will end up getting removed for lack of utility, and some of them will end up getting combined or transformed, but there should still be plenty of features to work with.

In [13]:
# combining Size and Screen Size, Imputing one value, deleting Size column, remove xx",
#     convert to float
df['Screen Size'].fillna(df['Size'], inplace=True)
df.loc[19,'Screen Size'] = '34"'
df = df.drop(['Size'], axis=1)
df['Screen Size'] = df['Screen Size'].str.replace('"', '')
df['Screen Size'] = df['Screen Size'].astype('float')


In [14]:
# changing ratings count of 1 to 0, convert to float.
df[['rating_count', 'avg_rating']] = df[['rating_count', 'avg_rating']].astype(float)
df[['rating_count', 'avg_rating']] = df[['rating_count',
                                         'avg_rating']].replace(to_replace=1, value=0)
type(df.rating_count[0])

numpy.float64

In [15]:
# changing rating average of 0 to Null
df['avg_rating']= df['avg_rating'].replace(to_replace=0, value='NaN')

In [16]:
# combining display type columns
df.loc[6,'Display Type'] = 'Nano IPS'
df['Display Type'].fillna(df['Display  Type'], inplace=True)
df.loc[19,'Display Type'] = 'Nano IPS'
df.drop(['Display  Type'], axis=1, inplace=True)

In [17]:
# combining color depth columns and imputing one value
df['Color Depth (Number of Colors)'].fillna(df['Color Depth(Number of Colors)'], inplace=True)
df.drop(['Color Depth(Number of Colors)'], axis=1, inplace=True)
df.loc[19,'Color Depth (Number of Colors)'] = '1.07B'

In [18]:
# combing color gamut columns -note, still 2 NaN values
df['Color Gamut (Typ.)'].fillna(df['Color Gamut'], inplace=True)
df.loc[19,'Color Gamut (Typ.)'] = 'DCI-P3 98% (CIE1976)'
df.loc[7,'Color Gamut (Typ.)'] = 'sRGB 99% Color Gamut'
df.drop(['Color Gamut'], axis=1, inplace=True)

In [19]:
# combining pixel pitch columns
df['Pixel Pitch (mm)'].fillna(df['Pixel Pitch(mm)'], inplace=True)
df.loc[19,'Pixel Pitch (mm)'] = '0.18159 x 0.18159 mm'
df['Pixel Pitch (mm)'] = df['Pixel Pitch (mm)'].str[:5].astype(float)
df.drop('Pixel Pitch(mm)', axis=1, inplace=True)

In [20]:
# clearing up Response time, more later
df['Response Time'].fillna(df['Response Time (GTG)'], inplace=True)
df.drop('Response Time (GTG)', axis=1, inplace=True)

In [21]:
#making Response time controll a bool integer value
df['Response Time Control'].fillna(0, inplace=True)
df['Response Time Control'].replace(to_replace='Yes', value=1, inplace=True)
df['Response Time Control'] = df['Response Time Control'].astype(int)

In [22]:
# df['Brightness'] - this column will need to be transformed into a useable format later

In [23]:
# df['Viewing Angle'] has no variation so it gets dropped
df.drop(['Viewing Angle'], axis=1, inplace=True)

In [24]:
# combining display port column and delete old col
df['Display Port'].fillna(df['Display Port Cable'], inplace=True)
df.drop('Display Port Cable', axis=1, inplace=True)



In [25]:
# add imputed value for display resolution
df.loc[19,'Display Resolution'] = '5K2K UW-UHD' 
df['Contrast Ratio'].fillna(df['Contrast Ratio (DFC)'], inplace=True)
df.drop('Contrast Ratio (DFC)', axis=1, inplace=True)

In [26]:
# convert HDMI to numeric
hdmi_map = {
    'Yes':1,
    'Yes x 2':2,
    'Yes (2.0 version)':1
}
df['HDMI'] = df['HDMI'].map(hdmi_map)

In [27]:
# converting display port to numeric and dropping duplicate column
display_port_map = {
    'Yes x 1':1,
    np.NaN:0,
    'Yes x 2':2,
    'Yes':1
}
df['DisplayPort'] = df['DisplayPort'].map(display_port_map)
df.drop('Display Port', axis=1, inplace=True)

In [28]:
#converting headphone to numeric
headphone_d = {
    'Yes':1,
    np.NaN:0}
df['Headphone Out'].replace(headphone_d, inplace=True)

In [29]:
#dropping columns that refer to power supply
drop_cols = df.columns[17:24]
df.drop(drop_cols, axis=1, inplace=True)

In [30]:
#setting up the freesync variables as different numerical columns

freesync_d = {
    np.NaN:0,
    'AMD FreeSync™':1,
    'RADEON FreeSync™':1
}
df['Adaptive Sync'].fillna(df['Adaptive-Sync (Radeon FreeSync™)'], inplace=True)

# setting up freesync column
df['freesync'] = 0
filter1 = df['Adaptive Sync'] == 'AMD FreeSync™'
filter2 = df['Adaptive Sync'] == 'RADEON FreeSync™'
df.loc[(filter1)|(filter2),'freesync'] = 1

# setting up freesync premium column
df['freesync_premium'] = 0
filter1 = df['Adaptive Sync'] == 'AMD FreeSync™ Premium'
df.loc[(filter1),'freesync_premium'] = 1

#setting up freesync premium pro column
df['freesync_premium_pro'] = 0
filter1 = df['Adaptive Sync'] == 'AMD FreeSync™ Premium Pro'
df.loc[(filter1),'freesync_premium_pro'] = 1
df['freesync_premium_pro']

df.drop(['Adaptive-Sync (Radeon FreeSync™)', 'Adaptive Sync'],axis=1, inplace=True)

In [31]:
#replacing value for spelling error column
df['FreeSync (Low Frame Compensation)'].fillna(
    df['FreeSync (Low Frame Conpensation)'], inplace=True)
df.drop(['FreeSync (Low Frame Conpensation)'], axis=1, inplace=True)

In [32]:
# cleaning up a bunch of yes/no columns
map_d = {
    np.NaN:0,
    'Yes':1
} 
df.iloc[:,17:27] = df.iloc[:,17:27].replace(map_d)

In [33]:
# dropping stand and size columns
size_drop_cols = df.iloc[:, 31:40].columns
df.drop(size_drop_cols, axis=1, inplace=True)

In [34]:
# combining color weakness columns
mask = df['Color Weakness']=='Yes'
df.loc[mask,'Color Weakness Mode'] = 1


# combining black stabilizer columns
mask = df['Black Stabilizer®']=='Yes'
df.loc[mask, 'Black Stabilizer'] = 1

# combining dynamic sync cols
mask = df['Dynamic Action Sync®']=='Yes'
df.loc[mask,'Dynamic Action Sync'] = 1

# combining Flickersafe cols
mask = df['Flicker Safe®']=='Yes'
df.loc[mask, 'Flicker Safe'] = 1

# combinbin super resolution columns
df['Super Resolution+'].fillna(0, inplace=True)
df['Super Resolution+'].replace(to_replace='Yes', value= 1, inplace=True)
mask1 = df['Super  Resolution +']=='Yes'
mask2 = df['Super Resolution +']=='Yes'
mask3 = df['Super+ Resolution']=='Yes'
df.loc[(mask1)|(mask2)|(mask3),'Super Resolution+'] = 1

#combining OSC columns
df['On Screen Control (OSC)'].fillna(0, inplace=True)
mask1 = (df['On Screen Control']=='Yes')
mask2 = (df['OnScreen Control (OSC)']=='Yes')
df.loc[(mask1)|(mask2),'On Screen Control (OSC)']=1

# combining HDR cols
df.loc[19, 'VESA DisplayHDR™'] = 'VESA HDR 600'
df['HDR Effect'].fillna(0, inplace=True)
df['HDR Effect'].replace(to_replace=['Yes'], value=['1'], inplace=True)
df['HDR'] = df['HDR Effect']
df.loc[[15,16,19],'HDR']=1

 # combine and replace speaker cols
replace = ['No Built-in Speaker', 'No-built in speaker', 'Yes']
r_with = [0, 0, 1]

df['Speaker'].fillna(0, inplace=True)
df['Speaker'].replace(to_replace=replace, value=r_with, inplace=True)

# converting response time col to single digit numerical
df['response_time_ms'] = df['Response Time'].str[0].astype(int).copy()

# imputed last moniter refresh rate
df.loc[19,'Refresh Rate'] = '75Hz'
df['Refresh Rate'] = df['Refresh Rate'].apply(lambda x: x.split(' ')[0])
df['refresh_rate(hz)'] = df['Refresh Rate'].str[0:-2].copy()

# make wide color gamut numerical
df['Wide Color Gamut'].fillna(0, inplace=True)
df['Wide Color Gamut'].replace(to_replace='Yes', value= 1, inplace=True)

# make 2020 model numeric
df['2020 Model'].fillna(0, inplace=True)
df['2020 Model'].replace({'Yes':1}, inplace=True)

# adding to nano ips col
mask = df['Nano IPS™ Technology']=='Yes'
df.loc[mask, 'Display Type'].replace(to_replace='IPS', value='Nano IPS', inplace=True)

# make blur col numeric 
df['1ms Motion Blur Reduction'].fillna(0, inplace=True)
df['1ms Motion Blur Reduction'].replace(to_replace='Yes', value=1, inplace=True)

# make curved numeric
df['Curved'].fillna(0, inplace=True)
df['Curved'].replace(to_replace='Yes', value=1, inplace=True)

# g-sync
df['NVIDIA G-Sync™'].fillna(0, inplace=True)
df['NVIDIA G-Sync™'].replace({'Yes (Compatible)':1,
                              'G-SYNC Compatible':1, 
                              'NVIDIA G-SYNC ULTIMATE':1},
                            inplace=True)

# filling na in Surface treatment
df['Surface Treatment'].fillna('N/A', inplace=True)
df['Surface Treatment']

# grouping resolution
res_d = {'4K UHD': '4K', 
         '5K2K UW-UHD':'UW-UHD',
         'UW QHD':'UW-QHD'}
df['Display Resolution'].replace(res_d, inplace=True)

# making price a float
df['price'] = df['price'].astype(float)

drop_cols = ['Color Weakness','Black Stabilizer®', 'Super  Resolution +',
             'Super Resolution +','Super+ Resolution',"Dual EDID (H/W)",
             'On Screen Control', 'OnScreen Control (OSC)', 'HDR (High Dynamic Range)',
             'HDR Effect','Windows 10','Country of origin','Country of Origin','Dynamic Action Sync®',
             'Flicker Safe®','D-Sub','Speaker Channel','Speaker Output', 'Response Time', 'VESA DSC', 
             'Active Dimming', 'Tilt (Angle)', 'Height (mm)','One Click Stand Set-up', 'Mouse Holder', 
             'Ergo Stand', 'Swivel (Angle)', 'Extend/Retract (mm)', 'USB-C (DP Alternate Mode)',
             'USB-C (Data Transsmission)', 'USB-C (Power Delivery)', 'USB Type C to A Gender', 'USB A to B',
             'DisplayPort OverClock', 'PBP (Picture by Picture)', 'Thunderbolt', 'Speaker Type', 'Active Dimming',
             'Thunderbolt 3', 'PBP', 'Rich Bass', 'Panel Type', 'USB-C', 'ThunderBolt', 'Normal On (EPA)', 
             'Output', 'Key Lock', 'DDC/CI', 'HDCP', 'Picture Mode', 'Factory Calibration', 'Automatic Standby', 
             'Six Axis Control', 'DAS Mode', 'Response Time Control', 'My Display Preset (in OSC)', 'Ratio',
             'Local Dimming', 'Wall Mount Size (mm)', 'Normal On (factory out condition Typ.)', 'Crosshair', 'USB', 
             'USB Type-C', 'Maxx Audio', 'USB Up-Stream (ver 3.0)', 'USB Down-Stream (ver 3.0)',
             'Product with Stand (WxHxD)', 'Product without Stand (WxHxD)','Shipping Size (WxHxD)',
             'Product Weight with Stand', 'Product Weight without Stand', 'Tilt(Angle)', 'Height(mm)', 'Pivot',
             'USB3.0 Upstream Cable', 'USB Up-stream', 'USB Down-stream', 'Overclock', 'Crosshair®',
             'Auto Input Switch', 'FPS Counter', 'Motion Blur Reduction Technology','User Define Key',
             'Power Save/Sleep Mode (Max.)', 'DC Off', 'DC Off (Max.)', 'H/W Calibration (True Color Pro)', 
             'Sphere Lighting', 'LG UltraGear™ Control Center', '4-Side Virtually Borderless Design', 
             'True Color Pro', 'Others (Accessory)', 'PIP', 'Game Mode', 'Auto Brightness', 
             'H/W Calibration Ready (True Color Pro)', 'Screen split2.0 (in OSC)', 'TUV-Type', 'UL(cUL)', 
             'FCC-B, CE', 'TCO 6.0', 'ErP', 'CCC (for China)', 'BSMI (for Taiwan)', 'Power Cord', 'Cable Holder',
             'Thunderbolt3 Cable', 'USB (USB B-to-A, White)', 'Nano IPS™ Technology',
             'FreeSync (Low Frame Compensation)', 'VESA DisplayHDR™', 'Refresh Rate']

df.drop(drop_cols, axis=1, inplace=True)

C:\Users\nate\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [35]:
# looking at what we have for null values
df.isnull().sum()

model_name                        0
model_title                       0
price                             0
rating_count                      0
avg_rating                        0
Screen Size                       0
Color Gamut (Typ.)                0
Aspect Ratio                      0
Display Resolution                0
Resolution                        0
Brightness                        0
Contrast Ratio                    0
Surface Treatment                 0
HDMI                              0
DisplayPort                       0
Headphone Out                     0
Color Weakness Mode               0
Black Stabilizer                  0
Dynamic Action Sync               0
Flicker Safe                      0
Smart Energy Saving               0
Reader Mode                       0
Color Calibrated                  0
Dual Controller                   0
On Screen Control (OSC)           0
Wide Color Gamut                  0
Color Depth (Number of Colors)    0
Pixel Pitch (mm)            

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   model_name                      20 non-null     object 
 1   model_title                     20 non-null     object 
 2   price                           20 non-null     float64
 3   rating_count                    20 non-null     float64
 4   avg_rating                      20 non-null     object 
 5   Screen Size                     20 non-null     float64
 6   Color Gamut (Typ.)              20 non-null     object 
 7   Aspect Ratio                    20 non-null     object 
 8   Display Resolution              20 non-null     object 
 9   Resolution                      20 non-null     object 
 10  Brightness                      20 non-null     object 
 11  Contrast Ratio                  20 non-null     object 
 12  Surface Treatment               20 non

# From data to DataFrame
I would have liked to have more rows here, but I have over 40 features for about 20 monitors to work with as I move into the last phase, DATA ANALYSIS. The webscraping was a little bit more tedious than I had hoped, and in the end I had to sacrifice number of monitors in order to get a more feature rich data set. I can always go back and try to scrape more data from a competitive brand, but to try to derive meaningful correlations from only 5 or 6 features across a lot more monitors didnt make a whole lot of sense for me. I have a few categorical variables to work with to see how much impact the feature has, but I also made sure I had plenty of numerical data to run some regression analysis with. There are still a few columns that I will probably end up manipulating as I start to explore the data more, but I have largely filtered out the irrelevant stuff and shaped the data into a useable form. I am going to save this dataframe as a CSV file and open it up in a new notebook so that if I have to rerun the notebook to clear the state I dont end up waiting too long. Excited to see which monitor emerges victorious!


In [37]:
df.to_csv('monitor_project.csv')